In [53]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
headers = ['brand','model', 'price', 'year', 'transmission', 'fuelConsumption', 'tax', 'mileage', 'fueltype', 'color', 'variant']
df = pd.DataFrame(columns = headers)


In [54]:
colors = ['beige', 'black', 'blue', 'bronze', 'brown', 'gold', 'green', 'grey', 'orange', 'purple', 'red', 'silver', 'turquoise', 'white', 'yellow']
cars = ['toyota', 'volvo', 'audi', 'bmw', 'citroen', 'ford', 'honda', 'hyundai', 'kia', 'nissan']
url = "https://www.exchangeandmart.co.uk/used-cars-for-sale/"

In [55]:
for color in colors:
    for pageNumber in range(1,100):
        for car in cars:
            currentUrl = url + car + "/any-distance-from-se15-1la/colour-" + color + '/page' + str(pageNumber)
            page = requests.get(currentUrl)
            soup = BeautifulSoup(page.text, 'html')
            items = soup.find_all('div', attrs={"class" : "result-item"})

            if items[0].find('span', attrs={"class" : "ttl_mk"}) is None:
                break
            
            for item in items:
                brand = item.find('span', attrs={"class" : "ttl_mk"}).text
                model = item.find('span', attrs={"class" : "ttl_md"}).text
                price = item.find('span', attrs={"class" : "price--primary"}).text
                variant = item.find('div', attrs={"class" : "result-item__variant"}).text
                keyDetails = item.find_all('p', attrs={"class" : "key-details__item"})
                if(len(keyDetails) < 6):
                    continue
                year = keyDetails[0].text
                transmission = keyDetails[1].text
                fuelConsumption = keyDetails[2].text
                tax = keyDetails[3].text
                mileage = keyDetails[4].text
                fueltype = keyDetails[5].text
                data = [brand, model, price, year, transmission, fuelConsumption, tax, mileage, fueltype, color, variant]
                df.loc[len(df)-1] = data

In [57]:
df.to_csv('AllData.csv')
groupedDFs = [v for k, v in df.groupby('brand')]
for brandDF in groupedDFs:
    brandDF.to_csv(''+brandDF.brand.iloc[0]+'.csv') 